In [509]:
import numpy as np
import pandas as pd
import re
import os
from io import StringIO

from sklearn.externals import joblib
from scipy.sparse import csr_matrix, lil_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
import sys
import time

%matplotlib inline

# Релизация модели FM

In [2]:
def RMSE_score(y_pred, y_real):
    return np.sqrt(np.sum(np.power(y_pred - y_real, 2)) / y_pred.shape[0])

def R2_score(y_pred, y_real):
    return 1 - np.sum(np.power(y_pred - y_real, 2)) / np.sum(np.power(y_real - np.mean(y_real), 2))

def R2_adj_score(y_pred, y_real, features):
    R2 = R2_score(y_pred, y_real)
    return 1 - (1 - R2) * (y_real.shape[0] - 1) / (y_real.shape[0] - features - 1)

In [617]:
class SGD2WAYFactorizationMachine(object):
    def __init__(self, 
                 k=5, 
                 epochs=5, 
                 batch_size = 64, 
                 epsilon=1e-4, 
                 step=0.001, 
                 step_V=None, 
                 lr_decay_rate=5e-7,
                 verbose=0, 
                 nobs_verbose=10
        ):
        self.epochs = epochs
        self.batch_size = batch_size
        self.epsilon = epsilon
        self.step = step
        self.verbose = verbose
        self.nobs_verbose = nobs_verbose
        self.lr_decay_rate = lr_decay_rate
        if step_V:
            self.stepV = step_V
        else:
            self.stepV = self.step / 10
        self.k = k
        
        self.w0 = None
        self.w1 = None
        self.V = None
        
        self.Z = None
        self.Z_squared = None
    
    def exp_decay(self, initial_lrate, epoch):
        k = self.lr_decay_rate
        lrate = initial_lrate * np.exp(-k*epoch)
        return lrate
        
    def fit(self, X, Y):
        self.w0 = 0
        self.w1 = np.random.normal(size=(X.shape[1], 1), scale=.1)
        self.V = np.random.normal(size=(X.shape[1], self.k), scale=.1)
        
        n = X.shape[0]
        timedelta = time.time()
        for epoch in range(2, self.epochs + 2):
            if self.verbose and epoch % self.nobs_verbose == 0:
                print("\r Progress: {}/{} ({}%), duration: {} sec".format(
                    epoch, self.epochs + 1, int(((epoch+1) / (self.epochs)) * 100), int(time.time() - timedelta)
                ), end="")
                timedelta = time.time()
            
            batch_indices = np.random.choice(n, self.batch_size)
            
            # densify sparse matrices
            x = X[batch_indices, :]
            if hasattr(X, 'todense'):
                x = np.asarray(x.todense())
            y = Y[batch_indices]
            
            y_hat = self.predict(x)
            dy = -2 * (y - y_hat)
            
            self.w0 -= np.mean(dy) * self.step
            self.w1 -= np.mean(dy * x, axis=0, keepdims=True).T * self.exp_decay(self.step, epoch)
            
            for f in range(self.k):
                left = x * self.Z[:, f].reshape(-1, 1)
                right = self.X_squared * self.V[:, f]
                self.V[:, f] -=  np.mean(dy * (left - right), axis=0) * self.exp_decay(self.stepV, epoch)
    
    def calculate_nonlinear_part(self, X):
        self.Z = X @ self.V
        self.X_squared = np.power(X, 2)
        return 1/2 * np.sum(
            np.power(self.Z, 2) - self.X_squared @ np.power(self.V, 2), 
            axis=1, keepdims=True
        )
        
    def predict(self, X):
        return self.w0 + X @ self.w1 + self.calculate_nonlinear_part(X)

# Проверка корректности работы алгоритма на примере искусственного датасета

# и сравнение с SGD регрессором sklearn

In [618]:
from sklearn.datasets import make_regression
from sklearn.linear_model import SGDRegressor

In [619]:
X, y, coefs = make_regression(n_samples=10000, n_features=4, n_targets=4, n_informative=2, coef=True)

In [620]:
X = np.concatenate([X, y[:, :-1]], axis=1)
y = y[:, -1].reshape(-1, 1)

In [621]:
fm = SGD2WAYFactorizationMachine(epochs=20000, batch_size=256, k=8, step=1e-5, step_V=1e-8, verbose=1, nobs_verbose=5000, lr_decay_rate=1e-5)
reg = SGDRegressor(fit_intercept=True, penalty='none', shuffle=False, learning_rate='constant', eta0=0.001, max_iter=10000)

In [622]:
reg.fit(X, y.squeeze())

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.001,
       fit_intercept=True, l1_ratio=0.15, learning_rate='constant',
       loss='squared_loss', max_iter=10000, n_iter=None, penalty='none',
       power_t=0.25, random_state=None, shuffle=False, tol=None, verbose=0,
       warm_start=False)

In [623]:
fm.fit(X, y)

 Progress: 20000/20001 (100%), duration: 3 sec

In [624]:
reg.coef_, reg.intercept_

(array([-2.65034225e+08,  1.74924163e+10, -1.91707160e+11, -3.20168751e+08,
        -4.60966195e+10, -4.30955970e+10, -3.49519559e+10]),
 array([5.38982709e+10]))

In [625]:
fm.w1, fm.w0, fm.V

(array([[ 0.0658891 ],
        [ 0.09245032],
        [ 0.06325088],
        [ 0.06153187],
        [ 0.08707379],
        [-0.16187898],
        [ 0.3424324 ]]),
 0.04277680891446344,
 array([[ 0.05304461,  0.04999643, -0.01406342,  0.05861141, -0.15165712,
          0.04917029,  0.09304436,  0.1058377 ],
        [ 0.07627837,  0.1543007 , -0.00335402, -0.17078368, -0.09273973,
         -0.02209483,  0.123709  ,  0.03692013],
        [-0.05447968,  0.01259196, -0.26783772, -0.08084042,  0.06597526,
         -0.0545381 , -0.10490917,  0.05618324],
        [-0.05429117,  0.08523016,  0.21162872, -0.03963514, -0.04638106,
         -0.00514878,  0.00390355,  0.10121871],
        [-0.01963095, -0.00914506, -0.01507281, -0.00844009,  0.00246768,
          0.00383982,  0.00084308, -0.02185324],
        [ 0.00655918,  0.07292702, -0.02914311, -0.00246394,  0.04966344,
          0.02354367,  0.02647275, -0.00898529],
        [-0.01503185,  0.01698133,  0.04201216,  0.00802643,  0.02300353,
   

In [626]:
print('предсказания полученные SGD регрессией')
reg.predict(X[:5])

предсказания полученные SGD регрессией


array([-1.28540890e+13,  9.19552143e+12,  1.23298966e+13,  2.18284721e+13,
        2.42433683e+13])

In [627]:
print('предсказания полученные FM')
fm.predict(X[:5])

предсказания полученные FM


array([[ -4.49029517],
       [-48.66375385],
       [-29.58515453],
       [-44.13095484],
       [-19.54679126]])

In [628]:
print('реальные значения таргета')
y[:5]

реальные значения таргета


array([[ -4.54589026],
       [-51.17486706],
       [-30.37219423],
       [-44.65509058],
       [-19.45926933]])

# Работа с данными

## Загрузка датасета

In [16]:
def process_document(doc):
    # movies ids
    film_ids = list(map(lambda x: int(x[:-1]), re.findall(r'\d+\:', doc)))
    # separate frames
    frames_raw = re.split(r'\d+\:', doc)
    frames_raw = frames_raw[1:]
    
    frames_totale = []

    for frame, movie_id in zip(frames_raw, film_ids):
        sub_df = pd.read_csv(StringIO(frame), names=['CustomerID','Rating','Date'])
        sub_df['MovieID'] = movie_id

        frames_totale.append(sub_df)

    dataset = pd.concat(frames_totale)
    
    return dataset

In [17]:
txtfile = None

In [18]:
if txtfile:
    del txtfile

with open('Task2/netflix-prize-data/combined_data_1.txt', 'r') as f:
    txtfile = f.read()
    
dataset_p1 = process_document(txtfile)

In [19]:
#if txtfile:
#    del txtfile
#
#with open('Task2/netflix-prize-data/combined_data_2.txt', 'r') as f:
#    txtfile = f.read()
#    
#dataset_p2 = process_document(txtfile)

In [73]:
dataset = dataset_p1
dataset = dataset[dataset.MovieID.isin(movie_table.index.unique().values)]

In [27]:
#dataset = pd.concat([dataset_p1, dataset_p2], axis=0)
#dataset = dataset.reset_index()

# Таблица с фичами для фильмов

In [28]:
txtfile = None

In [22]:
if txtfile:
    del txtfile
    
with open('Task2/netflix-prize-data/movie_titles.csv', 'r') as f:
    txtfile = f.read()

subs = re.compile(r'\d+,\d+,')
new_lines = []
for i in txtfile.split('\n'):
    sub_part = subs.sub('', i)
    if 'NULL' not in sub_part:
        if ',' not in sub_part:
            new_lines.append(i)
        else:
            left_part = i.replace(sub_part, '')
            right_part = sub_part
            new_lines.append(left_part + '"' + right_part + '"')
        
movie_info = '\n'.join(new_lines)
movie_table = pd.read_csv(StringIO(movie_info), names = ['MovieID', 'Year', 'Name'])

movie_table.MovieID = movie_table.MovieID.astype(int)
movie_table.Year = movie_table.Year.astype(int)

movie_table = movie_table.set_index('MovieID')

## Генерируем спарс матрицу

In [74]:
customer_alias = {j:i for i, j in enumerate(dataset.CustomerID.unique())}
movie_alias = {j:i for i, j in enumerate(dataset.MovieID.unique())}
year_alias = {j:i for i, j in enumerate(movie_table.Year.unique())}


f1_size = dataset.MovieID.unique().shape[0]
f2_size = dataset.CustomerID.unique().shape[0]
f3_size = movie_table.Year.unique().shape[0]

In [75]:
#sparse_dataset = csr_matrix((f1_size + f2_size + f3_size, dataset.size))
sparse_dataset = lil_matrix((dataset.shape[0], f1_size + f2_size + f3_size))

In [76]:
for j, i in enumerate(dataset.itertuples()):
    if j % 50000 == 0:
        print("\r Progress: {}/{} ({}%)".format(j, dataset.shape[0], int(((j+1) / dataset.shape[0]) * 100)), end="")
    #print(i)
    customer = customer_alias[i.CustomerID]
    movie = movie_alias[i.MovieID]
    year = year_alias[movie_table.loc[i.MovieID].Year]
    
    sparse_dataset[j, customer] = 1
    sparse_dataset[j, f1_size + movie] = 1
    sparse_dataset[j, f1_size + f2_size + year] = 1

 Progress: 24050000/24053575 (99%)

In [629]:
y = dataset.Rating.values

In [78]:
csr_sparse_dataset = csr_matrix(sparse_dataset)

In [81]:
joblib.dump(csr_sparse_dataset, 'sparse_df_1.bin')

['sparse_df_1.bin']

## FM на полной матрице

In [ ]:
rmse_train = []
rmse_test = []
r2_train = []
r2_test = []
r2_adj_train = []
r2_adj_test = []

kfold = KFold(n_splits=4)

count = 1
for ix_train, ix_test in kfold.split(csr_sparse_dataset):
    print('{}-th fold'.format(count))
    mdl = SGD2WAYFactorizationMachine(epochs=csr_sparse_dataset.shape[0] // 64, batch_size=128, k=2, step=.001, step_V=0.00001, verbose=True, nobs_verbose=100)
    
    X_train = csr_sparse_dataset[ix_train]
    y_train = y[ix_train].reshape(-1, 1)
    
    X_test = csr_sparse_dataset[ix_test]
    y_test = y[ix_test].reshape(-1, 1)
    
    mdl.fit(X_train, y_train)
    
    rmse_test.append(RMSE_score(mdl.predict(X_test), y_test))
    r2_test.append(R2_score(mdl.predict(X_test), y_test))
    r2_adj_test.append(R2_adj_score(mdl.predict(X_test), y_test, csr_sparse_dataset.shape[1]))
    
    rmse_train.append(RMSE_score(mdl.predict(X_train), y_train))
    r2_train.append(R2_score(mdl.predict(X_train), y_train))
    r2_adj_train.append(R2_adj_score(mdl.predict(X_train), y_train, csr_sparse_dataset.shape[1]))
    
    count += 1

1-th fold
 Progress: 100/375838 (0%), duration: 515 sec